In [1]:
import sqlite3
import pandas as pd
from src.models import *
import plotly.express as px
import numpy as np

### Get Data

In [2]:
with Session(engine) as session:
    query = (
        session
        .query(Event,TeamMemberEntry)
        .join(Event.team_member_entries, isouter=True)
    )
    print(query.statement)
    data = pd.read_sql(query.statement, engine)
data

SELECT event.no, event.playthrough_id_no, event.location_name, event.location_region, event.event_type, event.event_name, event.failed, event.round, team_member_entry.no AS no_1, team_member_entry.team_member_playthrough_id_no, team_member_entry.team_member_slot, team_member_entry.event_no, team_member_entry.level, team_member_entry.species, team_member_entry.dex_no, team_member_entry.type1, team_member_entry.type2 
FROM event LEFT OUTER JOIN team_member_entry ON event.no = team_member_entry.event_no


,no,playthrough_id_no,location_name,location_region,event_type,event_name,failed,round,no_1,team_member_playthrough_id_no,team_member_slot,event_no,level,species,dex_no,type1,type2
0,1,26852,Nuvema Town,Unova,Gift,Lillipup,0,0,1.0,26852,1.0,1.0,5.0,Lillipup,506.0,Normal,None
1,2,26852,Nuvema Town,Unova,Battle,PKMN Trainer Bianca,0,0,NaN,None,NaN,NaN,NaN,None,NaN,None,None
2,3,26852,Nuvema Town,Unova,Battle,PKMN Trainer Cheren,0,0,NaN,None,NaN,NaN,NaN,None,NaN,None,None
3,4,26852,Accumula Town,Unova,Battle,PKMN Trainer N,0,0,2.0,26852,1.0,4.0,7.0,None,NaN,None,None
4,5,26852,Route 2,Unova,Battle,Youngster Jimmy,0,0,3.0,26852,1.0,5.0,8.0,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,97,26852,Desert Resort,Unova,Battle,Psychic Low,0,0,NaN,None,NaN,NaN,NaN,None,NaN,None,None
105,98,26852,Desert Resort,Unova,Battle,PKMN Ranger Mylene,0,0,NaN,None,NaN,NaN,NaN,None,NaN,None,None
106,99,26852,Desert Resort,Unova,Battle,PKMN Ranger Jaden,0,0,57.0,26852,3.0,99.0,27.0,None,NaN,None,None
107,100,26852,Relic Castle,Unova,Battle,Psychic Perry,0,0,NaN,None,NaN,NaN,NaN,None,NaN,None,None


#### Level Data

In [3]:
df = (
    data
    .pivot_table(values='level',index='no',columns='team_member_slot')
    .reindex(np.arange(data['no'].min(),data['no'].max()+1))
    .reset_index()
    .ffill()
)
display(df)
px.line(df, x="no", y=[1,2,3]).update_layout(hovermode='x')

team_member_slot,no,1.0,2.0,3.0
0,1,5.0,NaN,NaN
1,2,5.0,NaN,NaN
2,3,5.0,NaN,NaN
3,4,7.0,NaN,NaN
4,5,8.0,NaN,NaN
...,...,...,...,...
96,97,27.0,26.0,26.0
97,98,27.0,26.0,26.0
98,99,27.0,26.0,27.0
99,100,27.0,26.0,27.0


## Levels Over Time

In [4]:
# px.line(data, x='')